In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

In [ ]:
md = pd.read_csv('/home/jvdzwaan/data/kb-ocr/metadata.csv', index_col=0)
md = md.set_index('file_prefix')
md

In [ ]:
performance = pd.read_csv('/home/jvdzwaan/data/kb-ocr/text_aligned_blocks-FR11-match_gs-20190522.csv', index_col=0)
#performance = pd.read_csv('/home/jvdzwaan/data/kb-ocr/FR11_match-gs_vs_GT_test-files_2019-05-22.csv', index_col=0)
performance

In [ ]:
def remove_out_and_page(row):
    #print(row.name)
    parts = row.name.split('_')
    return f'{parts[0]}_{parts[1]}'

performance['Identifier'] = performance.apply(lambda row: remove_out_and_page(row), axis=1)
performance = performance.set_index('Identifier')
performance

In [ ]:
baseline = pd.read_csv('/home/jvdzwaan/data/kb-ocr/text_aligned_blocks-match_gs-20190514.csv', index_col=0)
baseline.columns = ['CER_baseline', 'WER_baseline', 'WER (order independent)_baseline']
baseline['Identifier'] = baseline.apply(lambda row: remove_out_and_page(row), axis=1)
baseline = baseline.set_index('Identifier')
baseline

In [ ]:
result = baseline.join(performance).join(md)
result

In [ ]:
for n in ('CER', 'WER', 'WER (order independent)'):
    result[f'{n}_difference'] = result[f'{n}_baseline'] - result[f'{n}']
result

In [ ]:
result['CER'].describe()

In [ ]:
for group in result.groupby('ABBYY Version'):
    print('ABBYY', group[0])
    print(group[1]['WER (order independent)'].describe())

In [ ]:
for group in result.groupby('period'):
    print('Period', group[0])
    print(group[1]['CER'].describe())

In [ ]:
r = {}

for group in result.groupby(['ABBYY Version', 'period']):
    #print(f'ABBYY {group[0][0]}, period {group[0][1]}')
    d = group[1]['WER (order independent)'].describe()
    #print(d)
    #print(d['mean'])
    abby = group[0][0]
    p = int(group[0][1])
    b_label = f'{abby}_baseline'
    if b_label not in r.keys():
        r[b_label] = {}
    r[b_label][p] = group[1]['WER (order independent)_baseline'].describe()['mean']
    
    fr11_label = f'{abby}_FR11'
    if fr11_label not in r.keys():
        r[fr11_label] = {}
    r[fr11_label][p] = d['mean']
    
r = pd.DataFrame(r)
r.plot()

In [ ]:
result

In [ ]:
r2 = {}

for group in result.groupby(['ABBYY Version', 'period']):
    print(f'ABBYY {group[0][0]}, period {group[0][1]}')
    d = group[1]['WER_difference'].describe()
    #print(d)
    #print(d['mean'])
    abby = group[0][0]
    p = int(group[0][1])
    b_label = f'{abby}'
    if b_label not in r2.keys():
        r2[b_label] = {}
    r2[b_label][p] = group[1]['WER_difference'].describe()['mean']
r2 = pd.DataFrame(r2)
r2.plot()